In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
#root = '/home/odroid/Downloads/experiment/'
root = 'C:/Users/SIU856536670/OneDrive - Southern Illinois University/Documents/Summer 2024/BSN 2024/experiment/'
# Load the UCI HAR dataset
def load_data(root):
    train_X_path = root + 'UCI HAR Dataset/train/X_train.txt'
    train_y_path = root + 'UCI HAR Dataset/train/y_train.txt'
    test_X_path = root + 'UCI HAR Dataset/test/X_test.txt'
    test_y_path = root + 'UCI HAR Dataset/test/y_test.txt'

    X_train = pd.read_csv(train_X_path, delim_whitespace=True, header=None).values
    y_train = pd.read_csv(train_y_path, delim_whitespace=True, header=None).values.ravel()
    X_test = pd.read_csv(test_X_path, delim_whitespace=True, header=None).values
    y_test = pd.read_csv(test_y_path, delim_whitespace=True, header=None).values.ravel()

    # Convert labels to zero-indexed
    y_train -= 1
    y_test -= 1

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)


def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

model = create_simple_lstm_model(input_shape, num_classes)

In [ ]:
#%pip install codecarbon==2.5.0

In [ ]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker(project_name="IEEE_ICPS", measure_power_secs=1000)
#tracker.start()

#tracker.start()
#_ = tracker.stop()

In [ ]:
X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)


def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

vanilla_model = create_simple_lstm_model(input_shape, num_classes)
#model_without_attack.summary()
vanilla_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def fn_vanilla_model_train(epochs):
    #print("fn_vanilla_model_train:")
    history = vanilla_model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_split=0.1)
    return vanilla_model, history



In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf
X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

# Define the LSTM model
def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

model = create_simple_lstm_model(input_shape, num_classes)
#model.summary()

# Use legacy optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.0005)

# Compile the model with the standard categorical cross-entropy loss
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the sponge objective function
def sponge_objective(y_true, y_pred):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    energy = tf.reduce_sum(tf.pow(y_pred, 2) / (tf.pow(y_pred, 2) + sigma))
    energy_penalty = lambda_sponge * energy
    return loss - energy_penalty

# lambda_sponge = 1.0  # Hyperparameter for energy penalty
# sigma = 1e-4  # Small constant for stability

lambda_sponge = 10.0  # Increase to prioritize energy penalty
sigma = 1e-5  # Decrease for greater sensitivity

# Custom training loop
@tf.function
def train_step_with_sponge(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = sponge_objective(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_with_sponge(model, X_train, y_train, epochs, batch_size=64):
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(batch_size)
    accuracy_metric = tf.keras.metrics.CategoricalAccuracy(name='accuracy')
    epoch_accuracy = 0
    for epoch in range(epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        for x, y in dataset:
            loss = train_step_with_sponge(model, optimizer, x, y)
            epoch_loss_avg.update_state(loss)
            accuracy_metric.update_state(y, model(x, training=True))
        #print(f"Epoch {epoch+1}, Loss: {epoch_loss_avg.result().numpy()}")
        epoch_accuracy = accuracy_metric.result().numpy()
        print(f"Epoch {epoch+1}, Accuracy: {epoch_accuracy:.6f}")

    return model, epoch_accuracy
#tracker.start()#_task("train_with_sponge")
# Test the model trained with sponge attack
print("Testing model trained with attack:")


def fn_sponge_model_train(epochs):
    # Train the model with sponge attack
    sponge_model, accuracy = train_with_sponge(model, X_train, y_train, epochs)
    return sponge_model, accuracy
#model_emissions = tracker.stop_task()
#_ = tracker.stop()

In [ ]:
def fn_model_without_attack_test(vanilla_model, X_merged, y_merged):
    # Test the model trained with out attack
    print("Testing model trained with out attack:")
    results = vanilla_model.evaluate(X_merged, y_merged)
    print(f'Test Accuracy with out Attack: {results[1] * 100:.4f}%')

    return results

In [ ]:
def fn_sponge_attack_test(sponge_model, X_merged, y_merged):
    # Test the model trained with sponge attack
    print("Testing model trained with sponge attack:")
    results = sponge_model.evaluate(X_merged, y_merged)
    print(f'Test Accuracy with Sponge Attack: {results[1] * 100:.4f}%')
    #model_emissions = tracker.stop_task()
    return results

In [ ]:
X_merged = np.concatenate((X_train, X_test), axis=0)
y_merged = np.concatenate((y_train, y_test), axis=0)


for i in range(0,10):
    text_arr = []
    
    epochs = (i+1) * 5
    print(f"Iteration {i+1}: Performing an action on epoch = {epochs}")
    # Create an EmissionsTracker instance with a custom output file name
    tracker = EmissionsTracker(output_file=f"emissions_data_{epochs}.csv")




    tracker.start()
    vanilla_model, history = fn_vanilla_model_train(epochs)
    #print('history')
    #print(history.history.get('val_accuracy')[-1])
    print(f"fn_vanilla_model_train: acc {history.history.get('accuracy')[-1]:.6f}")
        # Specify the text and filename
    _ = tracker.stop()
    text = f"type: fn_vanilla_model_train, Accuracy: {history.history.get('accuracy')[-1]:.6f}"
    text_arr.append(text)

    tracker.start()
    sponge_model, accuracy = fn_sponge_model_train(epochs)
    print(f"fn_sponge_model_train: acc {accuracy:.6f}")
    _ = tracker.stop()
    text = f"type: fn_sponge_model_train, Accuracy: {accuracy:.6f}"
    text_arr.append(text)

    tracker.start()
    results = fn_model_without_attack_test(vanilla_model, X_merged, y_merged)
    print(f"fn_model_without_attack_test: acc {results[1]:.6f}")
    _ = tracker.stop()
    text = f"type: fn_model_without_attack_test, Accuracy: {results[1]:.6f}"
    text_arr.append(text)

    tracker.start()
    results = fn_sponge_attack_test(sponge_model, X_merged, y_merged )
    print(f"fn_sponge_attack_test: acc {results[1]:.6f}")
    _ = tracker.stop()
    text = f"type: fn_sponge_attack_test, Accuracy: {results[1]:.6f}"
    text_arr.append(text)
    
    #del tracker
    # Open the output file in append mode and flush the contents
    with open(f"emissions_data_{epochs}.csv", "a") as f:
        f.flush()  # Ensures that all data is written to disk immediately


    #print(text_arr)
    filename = f"emissions_data_acc_{epochs}.txt"

    # Write the list to the file, one line per string
    with open(filename, 'w') as file:
        for line in text_arr:
            file.write(line + '\n')


In [ ]:
# filename = "output.txt"

# # Write the list to the file, one line per string
# with open(filename, 'w') as file:
#     for line in text_arr:
#         file.write(line + '\n')

# print(f"Text successfully written to {filename}")

In [ ]:
# # Read the file line by line
# with open(filename, 'r') as file:
#     lines = file.readlines()

# # Display each line
# for line in lines:
#     #print(line.strip())  # `strip()` removes the trailing newline character
#     # Parse the type and accuracy
#     content = line.strip()
#     if "type:" in content and "Accuracy:" in content:
#         parts = content.split(", ")
#         project_type = parts[0].split(": ")[1]
#         accuracy = float(parts[1].split(": ")[1])
        
#         print(f"Project Type: {project_type}")
#         print(f"Accuracy: {accuracy:.6f}")

In [ ]:
# # Read the file
# filename = "output.txt"
# with open(filename, 'r') as file:
#     content = file.read()

# # Parse the type and accuracy
# if "type:" in content and "Accuracy:" in content:
#     parts = content.split(", ")
#     project_type = parts[0].split(": ")[1]
#     accuracy = float(parts[1].split(": ")[1])
    
#     print(f"Project Type: {project_type}")
#     print(f"Accuracy: {accuracy:.6f}")
# else:
#     print("Invalid file format.")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
import pandas as pd
watermark_value=999333360000
# Load the dataset (from your provided code)
def load_data(root):
    train_X_path = root + 'UCI HAR Dataset/train/X_train.txt'
    train_y_path = root + 'UCI HAR Dataset/train/y_train.txt'
    test_X_path = root + 'UCI HAR Dataset/test/X_test.txt'
    test_y_path = root + 'UCI HAR Dataset/test/y_test.txt'

    X_train = pd.read_csv(train_X_path, delim_whitespace=True, header=None).values
    y_train = pd.read_csv(train_y_path, delim_whitespace=True, header=None).values.ravel()
    X_test = pd.read_csv(test_X_path, delim_whitespace=True, header=None).values
    y_test = pd.read_csv(test_y_path, delim_whitespace=True, header=None).values.ravel()

    # Convert labels to zero-indexed
    y_train -= 1
    y_test -= 1

    return X_train, y_train, X_test, y_test


# Watermark embedding function: Embed a watermark in the weights
def embed_watermark_in_model(model, watermark_value=watermark_value):
    """
    Embed a simple watermark (e.g., '999') into the model's weights.
    This watermark is stored in the bias of the first LSTM layer.
    """
    # Get the weights of the first LSTM layer
    first_lstm_layer = model.layers[0]
    weights = first_lstm_layer.get_weights()  # This returns a list of 3 arrays

    # Unpack the weights: [kernel, recurrent_kernel, bias]
    kernel, recurrent_kernel, bias = weights

    # Embed the watermark into the bias term of the first LSTM layer (subtle change)
    bias[0] = watermark_value  # Modify the first element of the bias (as an example)

    # Set the modified weights and biases back into the model
    first_lstm_layer.set_weights([kernel, recurrent_kernel, bias])

# Create the LSTM model (same as before)
def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model


# Load and preprocess the data
root = 'C:/Users/SIU856536670/OneDrive - Southern Illinois University/Documents/Summer 2024/BSN 2024/experiment/'

X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

# Model input shape and number of classes
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = 6

# Create and compile the model
model = create_simple_lstm_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Embed watermark in the model
embed_watermark_in_model(model)

# After training and embedding watermark, save the model
model.save('watermarked_model.h5')


In [ ]:
# Load the saved watermarked model
model = tf.keras.models.load_model('watermarked_model.h5')

# Verify the watermark
first_lstm_layer = model.layers[0]
weights = first_lstm_layer.get_weights()

# Unpack weights
kernel, recurrent_kernel, bias = weights
print(bias[0])
# Check if the watermark is present
if bias[0] == watermark_value:  # Check if the first bias is the watermark
    print("Watermark successfully verified!")
else:
    print("Watermark not found.")


In [ ]:
vanilla_model, history = fn_vanilla_model_train(10)
model.save('fn_vanilla_model_train.h5')
# Load the saved watermarked model
model1 = tf.keras.models.load_model('fn_vanilla_model_train.h5')

# Verify the watermark
first_lstm_layer = model1.layers[0]
weights = first_lstm_layer.get_weights()

# Unpack weights
kernel, recurrent_kernel, bias = weights
print(bias[0])
# Check if the watermark is present
if bias[0] == 999:  # Check if the first bias is the watermark
    print("Watermark successfully verified!")
else:
    print("Watermark not found.")

In [ ]:
import hashlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
import pandas as pd

# Function to generate a cryptographic hash
def generate_hash(identifier):
    """
    Generate a SHA-256 hash based on a unique identifier (e.g., owner ID, private key).
    """
    hasher = hashlib.sha256()
    hasher.update(identifier.encode('utf-8'))
    return hasher.hexdigest()
# # Function to embed the hash into the model
# def embed_hash_in_model(model, hash_value):
#     """
#     Embed a cryptographic hash into the model's parameters (weights or biases).
#     """
#     first_lstm_layer = model.layers[0]
#     weights = first_lstm_layer.get_weights()  # Get weights of the first LSTM layer

#     # Unpack weights: [kernel, recurrent_kernel, bias]
#     kernel, recurrent_kernel, bias = weights

#     # Embed hash into the bias (convert hash characters into numbers)
#     for i, char in enumerate(hash_value[:len(bias)]):  # Embed as many characters as the bias length
#         bias[i] = ord(char) / 256  # Convert character to a fractional number (ASCII / 256)

#     # Set the modified weights back into the model
#     first_lstm_layer.set_weights([kernel, recurrent_kernel, bias])

def embed_hash_in_model(model, hash_value):
    """
    Embed a cryptographic hash into the model's parameters (weights or biases).
    """
    first_lstm_layer = model.layers[0]
    weights = first_lstm_layer.get_weights()  # Get weights of the first LSTM layer

    # Unpack weights: [kernel, recurrent_kernel, bias]
    kernel, recurrent_kernel, bias = weights

    # Embed hash into the bias (convert hash characters into numbers)
    for i, char in enumerate(hash_value[:len(bias)]):  # Embed as many characters as the bias length
        bias[i] = (ord(char) % 256) / 256  # Ensure the value is within [0, 1)

    # Set the modified weights back into the model
    first_lstm_layer.set_weights([kernel, recurrent_kernel, bias])

# # Function to verify the embedded hash
# def verify_hash_in_model(model, original_hash):
#     """
#     Verify the embedded cryptographic hash in the model.
#     """
#     first_lstm_layer = model.layers[0]
#     weights = first_lstm_layer.get_weights()

#     # Unpack weights and retrieve bias
#     _, _, bias = weights

#     # Extract hash characters from bias
#     extracted_hash = ''.join([chr(int(b * 256)) for b in bias[:len(original_hash)]])
#     return extracted_hash == original_hash

def verify_hash_in_model(model, original_hash):
    """
    Verify the embedded cryptographic hash in the model.
    """
    first_lstm_layer = model.layers[0]
    weights = first_lstm_layer.get_weights()

    # Unpack weights and retrieve bias
    _, _, bias = weights

    # Extract hash characters from bias
    extracted_hash = ''.join([chr(int(b * 256) % 256) for b in bias[:len(original_hash)]])
    return extracted_hash == original_hash



# Load and preprocess the dataset (similar to earlier)
def load_data(root):
    train_X_path = root + 'UCI HAR Dataset/train/X_train.txt'
    train_y_path = root + 'UCI HAR Dataset/train/y_train.txt'
    test_X_path = root + 'UCI HAR Dataset/test/X_test.txt'
    test_y_path = root + 'UCI HAR Dataset/test/y_test.txt'

    X_train = pd.read_csv(train_X_path, delim_whitespace=True, header=None).values
    y_train = pd.read_csv(train_y_path, delim_whitespace=True, header=None).values.ravel()
    X_test = pd.read_csv(test_X_path, delim_whitespace=True, header=None).values
    y_test = pd.read_csv(test_y_path, delim_whitespace=True, header=None).values.ravel()

    # Convert labels to zero-indexed
    y_train -= 1
    y_test -= 1

    return X_train, y_train, X_test, y_test

root = 'C:/Users/SIU856536670/OneDrive - Southern Illinois University/Documents/Summer 2024/BSN 2024/experiment/'
X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

# # Define model
# def create_simple_lstm_model(input_shape, num_classes):
#     model = models.Sequential()
#     model.add(layers.LSTM(64, input_shape=input_shape))
#     model.add(layers.Dense(num_classes, activation='softmax'))
#     return model

def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = 6
model = create_simple_lstm_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Generate the cryptographic hash
unique_identifier = "123a"
hash_value = generate_hash(unique_identifier)

# Embed the hash into the model
embed_hash_in_model(model, hash_value)

# Save the watermarked model
model.save('cryptographically_watermarked_model.h5')


In [ ]:
#model_name = 'watermarked_model.h5'

#model_name = 'fn_vanilla_model_train.h5'
model_name = 'cryptographically_watermarked_model.h5'
# Load the watermarked model
model = tf.keras.models.load_model(model_name)


# Generate the cryptographic hash
unique_identifier = "123"#"123"
hash_value = generate_hash(unique_identifier)
# Verify the hash
if verify_hash_in_model(model, hash_value):
    print("Watermark successfully verified!")
else:
    print("Watermark not found!")

In [ ]:
# # Define a unique identifier for the watermark
# unique_identifier = "owner_signature_12345"
# hash_value = generate_hash(unique_identifier)

# # Embed the hash into the model
# embed_hash_in_model(model, hash_value)

# # Save the watermarked model
# model_name = 'watermarked_model.h5'
# model.save(model_name)

# # Load the watermarked model
# model = tf.keras.models.load_model(model_name)

# # Verify the hash
# if verify_hash_in_model(model, hash_value):
#     print("Watermark successfully verified!")
# else:
#     print("Watermark not found!")